In [ ]:
from lxml import etree
import pandas as pd
from io import StringIO
import requests
from urllib.parse import quote_plus

In [ ]:
# url = 'https://www.fpds.gov/ezsearch/fpdsportal?s=FPDS&indexName=awardfull&templateName=1.5.2&q=LAST_MOD_DATE%3A%5B2020%2F04%2F02%2C2020%2F04%2F02%5D&rss=1&feed=atom0.3'
# url = 'https://www.fpds.gov/ezsearch/fpdsportal?s=FPDS.GOV&indexName=icdindex&templateName=1.5.2&q=+DEPARTMENT_NAME%3A%22General+Services+Administration%22&rss=1&feed=atom0.3'
# url = 'https://www.fpds.gov/ezsearch/search.do?indexName=awardfull&templateName=1.5.2&s=FPDS.GOV&q=financial+assistance+actions&rss=1&feed=atom0.3'
# url = 'https://www.fpds.gov/ezsearch/fpdsportal?s=FPDS.GOV&indexName=awardfull&templateName=1.5.2&rss=1&feed=atom0.3&q=+LAST_MOD_DATE%3A%5B2021%2F10%2F01%2C2021%2F11%2F01%5D'

base_url = 'https://www.fpds.gov/ezsearch/fpdsportal?s=FPDS.GOV&indexName=awardfull&templateName=1.5.2&rss=1&feed=atom0.3&q=+LAST_MOD_DATE'
start_date = '2021/10/01'
end_date = '2021/11/01'
param = f':[{start_date},{end_date}]'
url = base_url + quote_plus(param)

print(url)


In [ ]:
response = requests.get(url)
print(response.ok)

In [ ]:
doc = etree.XML(response.content)

In [ ]:
xslt_tree = etree.parse("fpds_feed.xsl")
transform = etree.XSLT(xslt_tree)

In [ ]:
result = transform(doc)

In [ ]:
csv_str =str(result)
# csv_str

In [ ]:
mystr = StringIO(csv_str)

df = pd.read_csv(mystr, sep="\t", header=None, index_col=0)
df.head(10)

In [ ]:
df_new = df.transpose(copy=True)
df_new.iloc[:, :10]

In [ ]:
df_new.shape

In [ ]:
# output dataframe to a .csv file
df_new.to_csv("fpds_feed_result.csv", index=False)

In [1]:
from fpds_feed import fpds_feed_reader

In [2]:
reader = fpds_feed_reader()


In [4]:
df = reader.get_data("2021/11/01", "2021/11/15")
df.head(10)

,1,2,3,4,5,6,7,8,9,10
0,,,,,,,,,,
agencyID,9700,9700,9700,9700,9700,9700,9700,9700,9700,9700
agencyName,DEPT OF DEFENSE,DEPT OF DEFENSE,DEPT OF DEFENSE,DEPT OF DEFENSE,DEPT OF DEFENSE,DEPT OF DEFENSE,DEPT OF DEFENSE,DEPT OF DEFENSE,DEPT OF DEFENSE,DEPT OF DEFENSE
PIID,00001,00001,00001,00002,00003,00003,00003,00003,00004,00004
modNumber,0,0,0,0,0,0,0,0,0,0
transactionNumber,1,2,3,1,1,2,3,3,1,2
ref-agencyID,9700,9700,9700,9700,9700,9700,9700,9700,9700,9700
ref-agencyName,DEPT OF DEFENSE,DEPT OF DEFENSE,DEPT OF DEFENSE,DEPT OF DEFENSE,DEPT OF DEFENSE,DEPT OF DEFENSE,DEPT OF DEFENSE,DEPT OF DEFENSE,DEPT OF DEFENSE,DEPT OF DEFENSE
ref-PIID,DAAG9999A0005,DAAG9999A0014,DAAG9999A0017,DAAG9999A0015,DAAG9999A0015,DAAG9999A0017,DAAG9999A0005,DAAG9999A0014,DAAG9999A0015,DAAG9999A0005
ref-modNumber,0,0,0,0,0,0,0,0,0,0
